In [9]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import model_from_json
from keras.models import Sequential
from sklearn import metrics
%matplotlib inline

In [2]:
df = pd.read_csv('../experiments/results.csv')
print(df)

    Unnamed: 0                             X_train_fname  \
0            0                       ../data/X_train.npy   
1            1                       ../data/X_train.npy   
2            2                       ../data/X_train.npy   
3            3                   ../data/X_train_192.npy   
4            4                   ../data/X_train_192.npy   
5            5                   ../data/X_train_192.npy   
6            6                   ../data/X_train_192.npy   
7            7          ../data/X_train_scaled_jaffe.npy   
8            8          ../data/X_train_scaled_jaffe.npy   
9            9  ../data/X_train_scaled_jaffe_ck_dlib.npy   
10          10  ../data/X_train_scaled_jaffe_ck_dlib.npy   

                               Y_train_fname  \
0                        ../data/Y_train.npy   
1                        ../data/Y_train.npy   
2                        ../data/Y_train.npy   
3                    ../data/Y_train_192.npy   
4                    ../data/Y_train_19

In [3]:
df['acc'] = df['acc'].apply(eval)
df['val_acc'] = df['val_acc'].apply(eval)

df['final_acc'] = df['acc'].map(lambda x: x[-1])
df['final_val_acc'] = df['val_acc'].map(lambda x: x[-1])
print(df[['model_id','final_acc','final_val_acc']])


    model_id  final_acc  final_val_acc
0          1   0.646437       0.550589
1          2   0.595287       0.557135
2          3   0.252618       0.242940
3          5   0.704414       0.577707
4          7   0.555171       0.483636
5          7   0.764073       0.514494
6          6   0.848326       0.523471
7          8   0.770586       0.510136
8          9   0.739759       0.498791
9         10   0.930421       0.549747
10        11   0.942228       0.552778


In [4]:
X_test = np.load('../data/X_test_192.npy')
Y_test = np.load('../data/Y_test_192.npy')
with open('../experiments/models/model_5.json') as f:
    model = model_from_json(f.read())

model.load_weights('../experiments/weights/weights_5.h5')

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.predict(X_test[:5]))
print(Y_test[:5])

[[  1.24409598e-05   7.97630548e-02   2.45165620e-06   3.20551480e-05
    9.20159876e-01   3.01857726e-05]
 [  6.86784506e-06   1.67836151e-05   9.99790490e-01   6.93609254e-05
    2.95443465e-06   1.13636270e-04]
 [  5.09581305e-02   1.25065699e-01   7.39918053e-02   2.72510886e-01
    4.08596769e-02   4.36613768e-01]
 [  1.42585704e-04   1.05219049e-04   9.98595893e-01   4.46506601e-04
    5.14308385e-06   7.04612874e-04]
 [  5.20860297e-08   8.77099637e-09   9.99995708e-01   1.57348978e-08
    3.24142380e-09   4.13465250e-06]]
[[ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]]


In [7]:
with open('../experiments/models/model_5.json') as f:
    model = model_from_json(f.read())

model.load_weights('../experiments/weights/weights_5.h5')

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 192, 192)      832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 192, 192)      25632     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 192, 192)      25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 96, 96)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 96, 96)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 96, 96)        36928     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 96, 96)        36928     
__________

In [8]:
with open('../experiments/models/model_11.json') as f:
    model = model_from_json(f.read())

model.load_weights('../experiments/weights/weights_11.h5')

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 192, 192)      832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 192, 192)      25632     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 192, 192)      25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 96, 96)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 96, 96)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 96, 96)        36928     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 96, 96)        36928     
__________

In [18]:
test = {'test_accuracy': [], 'con_mat': []}

for i in range(len(df)):
    #Load model
    model_id = df.iloc[i]['model_id']
    print('Model ID: {}'.format(model_id))
    
    with open('../experiments/models/model_{}.json'.format(model_id)) as f:
        model = model_from_json(f.read())

    model.load_weights('../experiments/weights/weights_{}.h5'.format(model_id))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    #Load test set
    X_train_fname = df.iloc[i]['X_train_fname']
    Y_train_fname = df.iloc[i]['Y_train_fname']
    X_test_fname = X_train_fname.replace('train', 'test')
    Y_test_fname = Y_train_fname.replace('train', 'test')
    X_test = np.load('../data/{}'.format(X_test_fname))
    Y_test = np.load('../data/{}'.format(Y_test_fname))
    Y_test = np.argmax(Y_test,axis=1)
    #Run prediction
    Y_pred = model.predict(X_test)
    Y_pred= np.argmax(Y_pred, axis=1)
    
    #get accuracy on test set
    con_mat = metrics.confusion_matrix(Y_test, Y_pred)
    acc = metrics.accuracy_score(Y_test, Y_pred)
              
    print(con_mat)
    print(acc)
              
    #load into dict
    test['test_accuracy'] = acc
    test['con_mat'] = con_mat
              
#append dict to df
df['test_accuracy'] = test['test_accuracy']
df['con_mat'] = test['con_mat']


Model ID: 1
[[ 623  204  169  425   51  254]
 [ 199  520  115  366  197  177]
 [ 111   92 2261  193   57  136]
 [ 220  184  150  944   46  419]
 [  37  173   81   66  899   67]
 [ 116  146  232  477   52  999]]
0.54512131262
Model ID: 2
[[ 725   74  160  479   81  207]
 [ 248  276  100  468  319  163]
 [  85   34 2261  234  103  133]
 [ 277   81  196  980   61  368]
 [  46   88   78   62  991   58]
 [ 148   46  225  579   59  965]]
0.540932099843
Model ID: 3
[[   0    0 1726    0    0    0]
 [   0    0 1574    0    0    0]
 [   0    0 2850    0    0    0]
 [   0    0 1963    0    0    0]
 [   0    0 1323    0    0    0]
 [   0    0 2022    0    0    0]]
0.24873450864
Model ID: 5
[[ 820  134  121  439   73  139]
 [ 223  456   81  430  252  132]
 [ 117   75 2254  191   83  130]
 [ 254  178  122 1066   58  285]
 [  38  131   67   65  972   50]
 [ 177   97  201  535   56  956]]
0.569383836621
Model ID: 7
[[ 659  182  163  468   91  163]
 [ 262  396  106  410  290  110]
 [ 140   89 2075  26

ValueError: Length of values does not match length of index